Goal: compare PBA50 BAUS 2020 TAZ output ('run182_taz_summaries_2020.csv') with other data sources, and scale taz_summaries if needed.

Referencing data sources:
- population: Census 2020 Decennial
- household: Census 2020 PUMS 1 Year
- housing units: Census 2020 PUMS 1 Year
- employment: ESRI Business Analyst, Census 2020 PUMS 1 Year

Methodology (refer to the next cell):
- fields do not need to modify: id_fields, land_fields.

- pop_fields, hh_fields, housing_fields:

    - first get totals from Census data by county. "Total attributes" to compare: total population (Census Decennial), total group quarters population (Census Decennial), total housing units (ACS 1-year), total households (ACS 1-year).

    - Compare the 4 total attributes with BAUS output county sums, calculate an adjustment ratio for each. If BAUS 2020 output is very close to Census 2020 numbers, then no adjustment is needed, done. If need to adjust, continue:

    - apply the adjust ratio of each total attribute to all the TAZs. E.g., TAZ 1450 in Marin County where ACS county TOTHH (105298) / BAUS county TOTHH (108118) = 0.973917. BAUS TAZ 1450 total hh = 2729, so, need to adjust by 2729 * 0.973917. Within each county, resolve rounding error by adjusting the TAZ with the largest pop/hh/housing count.

    - for each TAZ, adjust sub-totals proportionally, and resolve the rounding errors to the largest category. E.g., TAZ 145), BAUS output has 'HHINCQ1' 269, 'HHINCQ2' 434, 'HHINCQ3' 589, 'HHINCQ4' 1437; adjust the first three categories by * 0.973917, and calculate HHINCQ4 = TOTHH - sum(HHINCQ1, HHINCQ2, HHINCQ3). 

- emp_fields:

    - get TAZ level total employment from ESRI Business Analyst (running script https://github.com/BayAreaMetro/petrale/blob/main/applications/travel_model_lu_inputs/2015/Employment/summarize_BusinessData_by_TAZ_industry.R), and summarize to county-level total employment.
    
    - compare county-level total employment from ESRI with BAUS. If they are close, no adjustment is needed, done. If the descrapencies are large, continue with the following adjustment:

    - for total employment, apply county-level adjustment ratio to all TAZs within each county. Within each county, resolve rounding error by adjusting the TAZ with the largest employment.

    - for each TAZ, adjust sub-totals by employment category, and resolve the rounding errors to the largest category.


- empres_fields: 

    - get county-level total employed residents from PUMS persons file, based on "ESR".

    - compare PUMS data with BAUS county-level 'EMPRES'. If they are close, no adjustment is needed, done. If the descrapencies are large, continue with the following adjustment:

    - apply county-level adjustment ratio to all TAZs within each county. Within each county, resolve rounding error by adjusting the TAZ with the largest employment.

- density_fields: recalculate based on adjusted values.

In [1]:
#  categorize BAUS output TAZ table fields into groups and write out the relationship among fields

                  # ID fields
id_fields      = ['TAZ', 'SD', 'ZONE', 'COUNTY', 'COUNTY_NAME', 'county', 'county_name']
                 
                  # employment: sum('AGREMPN', 'FPSEMPN', 'HEREMPN', 'RETEMPN', 'MWTEMPN', 'OTHEMPN') = 'TOTEMP'
emp_fields     = ['AGREMPN', 'FPSEMPN', 'HEREMPN', 'RETEMPN', 'MWTEMPN', 'OTHEMPN', 'TOTEMP']

empres_fields = [# employed residents = total population * resident employed ratio?
                 'EMPRES']

                  # sum('HHPOP', 'GQPOP') = 'TOTPOP'
pop_fields     = ['HHPOP', 'GQPOP', 'TOTPOP',
                  # Share of the population age 62 or older = 'TOTPOP' * 62P_ratio
                  'SHPOP62P',
                  # age breakdown: sum ('AGE0004', 'AGE0519', 'AGE2044', 'AGE4564', 'AGE65P') = 'TOTPOP'
                  'AGE0004', 'AGE0519', 'AGE2044', 'AGE4564', 'AGE65P',
                  # gp breakdown: sum ('gq_type_univ', 'gq_type_mil', 'gq_type_othnon') = 'gq_tot_pop'
                  'gq_type_univ', 'gq_type_mil', 'gq_type_othnon', 'gq_tot_pop'
]

                  # household income breakdown: sum('HHINCQ1', 'HHINCQ2', 'HHINCQ3', 'HHINCQ4') = 'TOTHH'
hh_fields      = ['HHINCQ1', 'HHINCQ2', 'HHINCQ3', 'HHINCQ4',  'TOTHH',
                  # by hh size: sum('hh_size_1', 'hh_size_2', 'hh_size_3', 'hh_size_4_plus') = 'TOTHH'
                  'hh_size_1', 'hh_size_2', 'hh_size_3', 'hh_size_4_plus',
                  # by worker count: sum('hh_wrks_0', 'hh_wrks_1', 'hh_wrks_2', 'hh_wrks_3_plus') = 'TOTHH'
                  'hh_wrks_0', 'hh_wrks_1', 'hh_wrks_2', 'hh_wrks_3_plus',
                  # by with kids or not: sum('hh_kids_no', 'hh_kids_yes') = 'TOTHH'
                  'hh_kids_no', 'hh_kids_yes']

                  # housing units: sum('MFDU', 'SFDU') = 'RES_UNITS'
housing_fields = ['RES_UNITS', 'MFDU', 'SFDU']

land_fields    = ['TOTACRE', 'RESACRE_UNWEIGHTED', 'CIACRE_UNWEIGHTED', 'CIACRE', 'RESACRE']

                  # Area type designation
density_fields = ['AREATYPE',
                  # density_pop = tot pop/acre, density_emp = tot emp/acre, density = density_pop + density_emp 
                  'DENSITY_POP', 'DENSITY_EMP', 'DENSITY']

In [2]:
import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [3]:
# inputs

# BAUS output
BOX_DIR = 'C:\\Users\\{}\\Box\\Modeling and Surveys'.format(os.getenv('USERNAME'))
BAUS_PBA50_FBP_DIR = os.path.join(BOX_DIR, 'Urban Modeling', 'Bay Area UrbanSim', 'PBA50', 'Final Blueprint runs',
                                  'Final Blueprint (s24)', 'BAUS v2.25 - FINAL VERSION')
BAUS_2020_TAZ_FILE = os.path.join(BAUS_PBA50_FBP_DIR, 'run182_taz_summaries_2020.csv')

# Census 2020 decennial data
L_DIR = 'L:\\Application\\Model_One\\TransitRecovery\\land_use_preprocessing'
CENSUS_INPUT_DIR = os.path.join(L_DIR, 'census_data')
DEC_P1_FILE = os.path.join(CENSUS_INPUT_DIR, 'DECENNIALPL2020.P1-2022-05-06T201441.csv') # P1: total pop by race, will use 'total population'
DEC_P5_FILE = os.path.join(CENSUS_INPUT_DIR, 'DECENNIALPL2020.P5-2022-05-06T201358.csv') # P5: group quarters pop by major group quarters type (use 'total group quarter pop')

# Census 2020 ACS PUMS 1-year data
PUMS_H_FILE = os.path.join(CENSUS_INPUT_DIR, 'hbayarea20.csv')  # PUMS housing records, for housing units and total household count
PUMS_P_FILE = os.path.join(CENSUS_INPUT_DIR, 'pbayarea20.csv') 

# ERSI business data for employment, already summarized by TAZ and scaled to match regional control totals;
# produced by the script "https://github.com/BayAreaMetro/petrale/blob/main/applications/travel_model_lu_inputs/2015/Employment/summarize_BusinessData_by_TAZ_industry.R"
ESRI_EMP_TAZ_FILE = os.path.join(L_DIR, 'esri_business_analyst', 'BusinessData_2020_TAZ_industry.csv')

# outputs
taz_summaries_scaled = os.path.join(L_DIR, 'run182_taz_summaries_2020.csv')

## 1. population, household, housing unite - compare BAUS with Census

### 1.1 get data from Census: total population, total households, total housing units

In [4]:
# 1. total population from Decennial P1 table
tot_pop_dec_raw = pd.read_csv(DEC_P1_FILE)
# only keep the total pop data and transpose the table so that each row represents one county
tot_pop_dec_raw.set_index('Label (Grouping)', inplace=True)
tot_pop_dec = tot_pop_dec_raw.loc[
    tot_pop_dec_raw.index == 'Total:'].transpose().rename(columns={'Total:': 'TOTPOP_dec'}).reset_index()
tot_pop_dec.loc[:, 'COUNTY_NAME'] = tot_pop_dec['index'].apply(lambda x: x.replace(' County, California', ''))
tot_pop_dec.drop(columns=['index'], inplace=True)
display(tot_pop_dec)

Label (Grouping),TOTPOP_dec,COUNTY_NAME
0,"1,682,353",Alameda
1,"1,165,927",Contra Costa
2,"262,321",Marin
3,"138,019",Napa
4,"873,965",San Francisco
5,"764,442",San Mateo
6,"1,936,259",Santa Clara
7,"453,491",Solano
8,"488,863",Sonoma


In [5]:
# 2. total group quarter pop from Decennial P5 table
tot_gp_pop_dec_raw = pd.read_csv(DEC_P5_FILE)
# only keep the total gp pop and transpose the table so that each row represents one county
tot_gp_pop_dec_raw.set_index('Label (Grouping)', inplace=True)
tot_gp_pop_dec = tot_gp_pop_dec_raw.loc[
    tot_gp_pop_dec_raw.index == 'Total:'].transpose().rename(columns={'Total:': 'GQPOP_dec'}).reset_index()
tot_gp_pop_dec.loc[:, 'COUNTY_NAME'] = tot_gp_pop_dec['index'].apply(lambda x: x.replace(' County, California', ''))
tot_gp_pop_dec.drop(columns=['index'], inplace=True)
display(tot_gp_pop_dec)

Label (Grouping),GQPOP_dec,COUNTY_NAME
0,"53,833",Alameda
1,"11,255",Contra Costa
2,"7,743",Marin
3,"5,172",Napa
4,"27,892",San Francisco
5,"9,352",San Mateo
6,"39,607",Santa Clara
7,"11,137",Solano
8,"8,866",Sonoma


In [6]:
# 3. total household from PUMS 2020

hh_pums_2020 = pd.read_csv(PUMS_H_FILE, usecols = [
    'PUMA', 
    'County_Name',
    'WGTP',    # Housing Unit Weight: 
#               0       Group quarters place holder record 
#               1..9999 Integer weight of housing unit
    'NP',      # Number of persons in this household:
#               0 .Vacant unit 
#               1 .One person in household or any person in group quarters 
#               2..20 .Number of persons in household 
    'TYPEHUGQ' # Type of unit 
#               1 .Housing unit
#               2 .Institutional group quarters 
#               3 .Noninstitutional group quarters 
])

# total households are represented by PUMS records with 'NP' > 0 (non-vacant) and 'WGTP' > 0 (non group quarter)
tot_hh_pums = hh_pums_2020.loc[(hh_pums_2020.NP > 0)].groupby('County_Name')['WGTP'].sum().reset_index()

tot_hh_pums.rename(columns={'County_Name': 'COUNTY_NAME', 
                            'WGTP': 'TOTHH_pums'}, inplace=True)

print(tot_hh_pums['TOTHH_pums'].sum())
display(tot_hh_pums)

2725688


,COUNTY_NAME,TOTHH_pums
0,Alameda,565858
1,Contra Costa,411225
2,Marin,105612
3,Napa,47926
4,San Francisco,346357
5,San Mateo,263750
6,Santa Clara,637782
7,Solano,155639
8,Sonoma,191539


In [7]:
# 4. total housing units from PUMS 2020
# PUMS records with unit type = 1 (non group quarter)
tot_unit_pums = hh_pums_2020.loc[hh_pums_2020.TYPEHUGQ == 1].groupby('County_Name')['WGTP'].sum().reset_index()
tot_unit_pums.rename(columns={'County_Name': 'COUNTY_NAME', 
                              'WGTP': 'RES_UNITS_pums'}, inplace=True)

print(tot_unit_pums['RES_UNITS_pums'].sum())
tot_unit_pums

2949825


,COUNTY_NAME,RES_UNITS_pums
0,Alameda,610507
1,Contra Costa,421676
2,Marin,112972
3,Napa,55639
4,San Francisco,402940
5,San Mateo,288145
6,Santa Clara,683596
7,Solano,164388
8,Sonoma,209962


In [8]:
# combine census county-level total metrics and modify county names to be consistent with modeling convention
census_tots_county = tot_pop_dec.merge(
                     tot_gp_pop_dec, on='COUNTY_NAME', how='outer').merge(
                     tot_hh_pums, on='COUNTY_NAME', how='outer').merge(
                     tot_unit_pums, on='COUNTY_NAME', how='outer')
# census_tots_county.loc[:, 'COUNTY_NAME'] = census_tots_county['index'].apply(lambda x: x.replace(' County, California', ''))
# census_tots_county.drop(columns='index', inplace=True)
# convert value fields to numeric
for col_name in ['TOTPOP_dec', 'GQPOP_dec']:
    census_tots_county.loc[:, col_name] = census_tots_county[col_name].apply(lambda x: int(x.replace(',','')))
display(census_tots_county)

,TOTPOP_dec,COUNTY_NAME,GQPOP_dec,TOTHH_pums,RES_UNITS_pums
0,1682353,Alameda,53833,565858,610507
1,1165927,Contra Costa,11255,411225,421676
2,262321,Marin,7743,105612,112972
3,138019,Napa,5172,47926,55639
4,873965,San Francisco,27892,346357,402940
5,764442,San Mateo,9352,263750,288145
6,1936259,Santa Clara,39607,637782,683596
7,453491,Solano,11137,155639,164388
8,488863,Sonoma,8866,191539,209962


### 1.2 compare PAB50 county-level total population, gp population, total households, total housing units with Census

In [9]:
# PBA50 BAUS 2020 output
baus_taz = pd.read_csv(BAUS_2020_TAZ_FILE)
taz_fields = list(baus_taz)
print('read {} rows of BAUS output taz data, with the following fields: {}'.format(baus_taz.shape[0], taz_fields))

read 1454 rows of BAUS output taz data, with the following fields: ['TAZ', 'SD', 'ZONE', 'COUNTY', 'COUNTY_NAME', 'AGREMPN', 'FPSEMPN', 'HEREMPN', 'RETEMPN', 'MWTEMPN', 'OTHEMPN', 'TOTEMP', 'HHINCQ1', 'HHINCQ2', 'HHINCQ3', 'HHINCQ4', 'HHPOP', 'TOTHH', 'SHPOP62P', 'GQPOP', 'TOTACRE', 'TOTPOP', 'RES_UNITS', 'MFDU', 'SFDU', 'RESACRE_UNWEIGHTED', 'CIACRE_UNWEIGHTED', 'CIACRE', 'RESACRE', 'EMPRES', 'DENSITY_POP', 'DENSITY_EMP', 'DENSITY', 'AREATYPE', 'AGE0004', 'AGE0519', 'AGE2044', 'AGE4564', 'AGE65P', 'gq_type_univ', 'gq_type_mil', 'gq_type_othnon', 'gq_tot_pop', 'hh_size_1', 'hh_size_2', 'hh_size_3', 'hh_size_4_plus', 'county', 'county_name', 'hh_wrks_0', 'hh_wrks_1', 'hh_wrks_2', 'hh_wrks_3_plus', 'hh_kids_no', 'hh_kids_yes']


In [10]:
# county-level sums of the same fields
baus_demo_tots_county = baus_taz.groupby('COUNTY_NAME')[['TOTPOP', 'GQPOP', 'TOTHH', 'RES_UNITS']].sum().reset_index()
display(baus_demo_tots_county.head())
baus_demo_tots_county.columns = ['COUNTY_NAME'] + [x+'_baus' for x in list(baus_demo_tots_county)[1:]]
display(baus_demo_tots_county.head())

,COUNTY_NAME,TOTPOP,GQPOP,TOTHH,RES_UNITS
0,Alameda,1664631.0,35275,574580.0,621497.0
1,Contra Costa,1144255.0,10032,388126.0,418647.0
2,Marin,272379.0,4363,108118.0,113274.0
3,Napa,149525.0,3861,51267.0,58307.0
4,San Francisco,948045.0,25007,388605.0,418990.0


,COUNTY_NAME,TOTPOP_baus,GQPOP_baus,TOTHH_baus,RES_UNITS_baus
0,Alameda,1664631.0,35275,574580.0,621497.0
1,Contra Costa,1144255.0,10032,388126.0,418647.0
2,Marin,272379.0,4363,108118.0,113274.0
3,Napa,149525.0,3861,51267.0,58307.0
4,San Francisco,948045.0,25007,388605.0,418990.0


In [11]:
baus_taz.groupby('COUNTY_NAME')['gq_tot_pop'].sum()

COUNTY_NAME
Alameda          27432.0
Contra Costa      5523.0
Marin             3072.0
Napa              2305.0
San Francisco    21091.0
San Mateo         5925.0
Santa Clara      23503.0
Solano            2822.0
Sonoma            6898.0
Name: gq_tot_pop, dtype: float64

In [12]:
tot_gp_pop_dec.groupby('COUNTY_NAME')['GQPOP_dec'].sum()

COUNTY_NAME
Alameda          53,833
Contra Costa     11,255
Marin             7,743
Napa              5,172
San Francisco    27,892
San Mateo         9,352
Santa Clara      39,607
Solano           11,137
Sonoma            8,866
Name: GQPOP_dec, dtype: object

In [13]:
# merge with census data
baus_census_tots_county_comp = baus_demo_tots_county.merge(census_tots_county, on='COUNTY_NAME', how='outer')

In [14]:
# calculate HHPOP
baus_census_tots_county_comp['HHPOP_dec'] = baus_census_tots_county_comp['TOTPOP_dec'] - baus_census_tots_county_comp['GQPOP_dec']
baus_census_tots_county_comp['HHPOP_baus'] = baus_census_tots_county_comp['TOTPOP_baus'] - baus_census_tots_county_comp['GQPOP_baus']

In [15]:
# calculate diffs and adjustment ratios
attr_source = {'TOTPOP': 'dec',
               'GQPOP' : 'dec',
               'HHPOP' : 'dec',
               'TOTHH' : 'pums',
               'RES_UNITS' : 'pums'}


for demo_attr in ['TOTPOP', 'GQPOP', 'HHPOP', 'TOTHH', 'RES_UNITS']:
    source = attr_source[demo_attr]
    baus_census_tots_county_comp.loc[:, demo_attr+'_'+source+'_baus_diff'] = \
        baus_census_tots_county_comp[demo_attr+'_'+source] - baus_census_tots_county_comp[demo_attr+'_baus']
    baus_census_tots_county_comp.loc[:, demo_attr+'_'+source+'_baus_ratio'] = \
        baus_census_tots_county_comp[demo_attr+'_'+source] / baus_census_tots_county_comp[demo_attr+'_baus']

In [16]:
# print out comparison

print('Total Population comparison:')
print('Census: {:,}'.format(baus_census_tots_county_comp['TOTPOP_dec'].sum()))
print('BAUS: {:,}'.format(int(baus_census_tots_county_comp['TOTPOP_baus'].sum())))
display(baus_census_tots_county_comp[[
    'COUNTY_NAME', 'TOTPOP_dec', 'TOTPOP_baus',
    'TOTPOP_dec_baus_diff', 'TOTPOP_dec_baus_ratio']].sort_values('TOTPOP_dec_baus_diff', ascending=False))

print('\nGroup quarters Population comparison:')
print('Census: {:,}'.format(baus_census_tots_county_comp['GQPOP_dec'].sum()))
print('BAUS: {:,}'.format(int(baus_census_tots_county_comp['GQPOP_baus'].sum())))
display(baus_census_tots_county_comp[[
    'COUNTY_NAME', 'GQPOP_dec', 'GQPOP_baus', 
    'GQPOP_dec_baus_diff', 'GQPOP_dec_baus_ratio']].sort_values('GQPOP_dec_baus_diff', ascending=False))

print('\nHouseholds Population comparison:')
print('Census: {:,}'.format(baus_census_tots_county_comp['HHPOP_dec'].sum()))
print('BAUS: {:,}'.format(int(baus_census_tots_county_comp['HHPOP_baus'].sum())))
display(baus_census_tots_county_comp[[
    'COUNTY_NAME', 'HHPOP_dec', 'HHPOP_baus',
    'HHPOP_dec_baus_diff', 'HHPOP_dec_baus_ratio']].sort_values('HHPOP_dec_baus_diff', ascending=False))


print('\nTotal Households comparison')
print('Census: {:,}'.format(baus_census_tots_county_comp['TOTHH_pums'].sum()))
print('BAUS: {:,}'.format(int(baus_census_tots_county_comp['TOTHH_baus'].sum())))
display(baus_census_tots_county_comp[[
    'COUNTY_NAME', 'TOTHH_pums', 'TOTHH_baus',
    'TOTHH_pums_baus_diff', 'TOTHH_pums_baus_ratio']].sort_values('TOTHH_pums_baus_diff', ascending=False))


print('\nTotal Housing Units comparison')
print('Census: {:,}'.format(baus_census_tots_county_comp['RES_UNITS_pums'].sum()))
print('BAUS: {:,}'.format(int(baus_census_tots_county_comp['RES_UNITS_baus'].sum())))
display(baus_census_tots_county_comp[[
    'COUNTY_NAME', 'RES_UNITS_pums', 'RES_UNITS_baus',
    'RES_UNITS_pums_baus_diff', 'RES_UNITS_pums_baus_ratio']].sort_values('RES_UNITS_pums_baus_diff', ascending=False))

Total Population comparison:
Census: 7,765,640
BAUS: 7,934,725


,COUNTY_NAME,TOTPOP_dec,TOTPOP_baus,TOTPOP_dec_baus_diff,TOTPOP_dec_baus_ratio
1,Contra Costa,1165927,1144255.0,21672.0,1.018940
0,Alameda,1682353,1664631.0,17722.0,1.010646
7,Solano,453491,439484.0,14007.0,1.031871
2,Marin,262321,272379.0,-10058.0,0.963074
3,Napa,138019,149525.0,-11506.0,0.923050
8,Sonoma,488863,515766.0,-26903.0,0.947839
5,San Mateo,764442,793473.0,-29031.0,0.963413
6,Santa Clara,1936259,2007167.0,-70908.0,0.964673
4,San Francisco,873965,948045.0,-74080.0,0.921860



Group quarters Population comparison:
Census: 174,857
BAUS: 140,730


,COUNTY_NAME,GQPOP_dec,GQPOP_baus,GQPOP_dec_baus_diff,GQPOP_dec_baus_ratio
0,Alameda,53833,35275,18558,1.526095
6,Santa Clara,39607,30006,9601,1.319969
2,Marin,7743,4363,3380,1.774696
4,San Francisco,27892,25007,2885,1.115368
3,Napa,5172,3861,1311,1.339549
1,Contra Costa,11255,10032,1223,1.121910
5,San Mateo,9352,9004,348,1.038649
8,Sonoma,8866,10081,-1215,0.879476
7,Solano,11137,13101,-1964,0.850088



Households Population comparison:
Census: 7,590,783
BAUS: 7,793,995


,COUNTY_NAME,HHPOP_dec,HHPOP_baus,HHPOP_dec_baus_diff,HHPOP_dec_baus_ratio
1,Contra Costa,1154672,1134223.0,20449.0,1.018029
7,Solano,442354,426383.0,15971.0,1.037457
0,Alameda,1628520,1629356.0,-836.0,0.999487
3,Napa,132847,145664.0,-12817.0,0.912010
2,Marin,254578,268016.0,-13438.0,0.949861
8,Sonoma,479997,505685.0,-25688.0,0.949202
5,San Mateo,755090,784469.0,-29379.0,0.962549
4,San Francisco,846073,923038.0,-76965.0,0.916618
6,Santa Clara,1896652,1977161.0,-80509.0,0.959281



Total Households comparison
Census: 2,725,688
BAUS: 2,764,643


,COUNTY_NAME,TOTHH_pums,TOTHH_baus,TOTHH_pums_baus_diff,TOTHH_pums_baus_ratio
1,Contra Costa,411225,388126.0,23099.0,1.059514
7,Solano,155639,143031.0,12608.0,1.088149
8,Sonoma,191539,188731.0,2808.0,1.014878
2,Marin,105612,108118.0,-2506.0,0.976822
3,Napa,47926,51267.0,-3341.0,0.934831
5,San Mateo,263750,271093.0,-7343.0,0.972913
0,Alameda,565858,574580.0,-8722.0,0.984820
6,Santa Clara,637782,651092.0,-13310.0,0.979557
4,San Francisco,346357,388605.0,-42248.0,0.891283



Total Housing Units comparison
Census: 2,949,825
BAUS: 2,987,471


,COUNTY_NAME,RES_UNITS_pums,RES_UNITS_baus,RES_UNITS_pums_baus_diff,RES_UNITS_pums_baus_ratio
7,Solano,164388,158865.0,5523.0,1.034765
1,Contra Costa,421676,418647.0,3029.0,1.007235
5,San Mateo,288145,286967.0,1178.0,1.004105
2,Marin,112972,113274.0,-302.0,0.997334
8,Sonoma,209962,211261.0,-1299.0,0.993851
3,Napa,55639,58307.0,-2668.0,0.954242
0,Alameda,610507,621497.0,-10990.0,0.982317
4,San Francisco,402940,418990.0,-16050.0,0.961694
6,Santa Clara,683596,699663.0,-16067.0,0.977036


In [17]:
baus_census_tots_county_comp

,COUNTY_NAME,TOTPOP_baus,GQPOP_baus,TOTHH_baus,RES_UNITS_baus,TOTPOP_dec,GQPOP_dec,TOTHH_pums,RES_UNITS_pums,HHPOP_dec,...,TOTPOP_dec_baus_diff,TOTPOP_dec_baus_ratio,GQPOP_dec_baus_diff,GQPOP_dec_baus_ratio,HHPOP_dec_baus_diff,HHPOP_dec_baus_ratio,TOTHH_pums_baus_diff,TOTHH_pums_baus_ratio,RES_UNITS_pums_baus_diff,RES_UNITS_pums_baus_ratio
0,Alameda,1664631.0,35275,574580.0,621497.0,1682353,53833,565858,610507,1628520,...,17722.0,1.010646,18558,1.526095,-836.0,0.999487,-8722.0,0.984820,-10990.0,0.982317
1,Contra Costa,1144255.0,10032,388126.0,418647.0,1165927,11255,411225,421676,1154672,...,21672.0,1.018940,1223,1.121910,20449.0,1.018029,23099.0,1.059514,3029.0,1.007235
2,Marin,272379.0,4363,108118.0,113274.0,262321,7743,105612,112972,254578,...,-10058.0,0.963074,3380,1.774696,-13438.0,0.949861,-2506.0,0.976822,-302.0,0.997334
3,Napa,149525.0,3861,51267.0,58307.0,138019,5172,47926,55639,132847,...,-11506.0,0.923050,1311,1.339549,-12817.0,0.912010,-3341.0,0.934831,-2668.0,0.954242
4,San Francisco,948045.0,25007,388605.0,418990.0,873965,27892,346357,402940,846073,...,-74080.0,0.921860,2885,1.115368,-76965.0,0.916618,-42248.0,0.891283,-16050.0,0.961694
5,San Mateo,793473.0,9004,271093.0,286967.0,764442,9352,263750,288145,755090,...,-29031.0,0.963413,348,1.038649,-29379.0,0.962549,-7343.0,0.972913,1178.0,1.004105
6,Santa Clara,2007167.0,30006,651092.0,699663.0,1936259,39607,637782,683596,1896652,...,-70908.0,0.964673,9601,1.319969,-80509.0,0.959281,-13310.0,0.979557,-16067.0,0.977036
7,Solano,439484.0,13101,143031.0,158865.0,453491,11137,155639,164388,442354,...,14007.0,1.031871,-1964,0.850088,15971.0,1.037457,12608.0,1.088149,5523.0,1.034765
8,Sonoma,515766.0,10081,188731.0,211261.0,488863,8866,191539,209962,479997,...,-26903.0,0.947839,-1215,0.879476,-25688.0,0.949202,2808.0,1.014878,-1299.0,0.993851


### 1.3 scale BAUS household counts to be consistent with ESRI data at the county level
Including total households and household by categories in each TAZ.

In [18]:
# calculate scale ratio by county
hh_adjust_ratio = baus_census_tots_county_comp[['COUNTY_NAME', 'TOTHH_pums_baus_ratio']]

baus_taz_hh_unscaled = baus_taz[['TAZ', 'COUNTY_NAME'] + hh_fields].merge(hh_adjust_ratio, on='COUNTY_NAME', how='left')
baus_taz_hh_unscaled

,TAZ,COUNTY_NAME,HHINCQ1,HHINCQ2,HHINCQ3,HHINCQ4,TOTHH,hh_size_1,hh_size_2,hh_size_3,hh_size_4_plus,hh_wrks_0,hh_wrks_1,hh_wrks_2,hh_wrks_3_plus,hh_kids_no,hh_kids_yes,TOTHH_pums_baus_ratio
0,1,San Francisco,23.0,11.0,6.0,10.0,50.0,23.0,25.0,2.0,0.0,19.0,25.0,6.0,0.0,46.0,4.0,0.891283
1,2,San Francisco,72.0,35.0,18.0,14.0,139.0,50.0,50.0,21.0,18.0,60.0,66.0,11.0,2.0,130.0,9.0,0.891283
2,3,San Francisco,129.0,81.0,32.0,26.0,268.0,145.0,84.0,22.0,17.0,112.0,126.0,23.0,7.0,249.0,19.0,0.891283
3,4,San Francisco,93.0,49.0,25.0,26.0,193.0,88.0,93.0,6.0,6.0,76.0,92.0,20.0,5.0,180.0,13.0,0.891283
4,5,San Francisco,324.0,152.0,69.0,65.0,610.0,394.0,135.0,42.0,39.0,251.0,290.0,54.0,15.0,570.0,40.0,0.891283
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1449,1450,Marin,269.0,434.0,589.0,1437.0,2729.0,609.0,992.0,492.0,636.0,568.0,717.0,1137.0,307.0,1886.0,843.0,0.976822
1450,1451,Marin,217.0,321.0,486.0,998.0,2022.0,447.0,712.0,390.0,473.0,330.0,766.0,700.0,226.0,1381.0,641.0,0.976822
1451,1452,Marin,269.0,393.0,438.0,1113.0,2213.0,544.0,800.0,366.0,503.0,472.0,855.0,734.0,152.0,1544.0,669.0,0.976822
1452,1453,Marin,77.0,54.0,53.0,113.0,297.0,71.0,135.0,43.0,48.0,42.0,80.0,145.0,30.0,222.0,75.0,0.976822


In [19]:
def scale_by_taz(df_unscaled, target_df, attr_name, attr_name_in_target_df, scale_ratio_field):
    """
    Scales a TAZ-level attribute (e.g. TOTHH, totemp) by county-level scale ratios, 
    so that the sums by county equal the target.
    Three steps:
     - apply the county-level scale ratio to all TAZs within each county
     - round to the nearest interger
     - within each county, correct rounding error by modifying the value of the largest TAZ 
       (largest in terms of the attribute for scaling)
    
    Arguments:
        df_unscaled:            e.g. baus_taz_hh_unadjusted
        target_df:              e.g. tot_hh_pums
        attr_name:              e.g. 'TOTHH'
        attr_name_in_target_df: e.g. 'TOTHH_pums'
        scale_ratio_field:      e.g. 'TOTHH_pums_baus_ratio'
    """
    
    df_scaled = pd.DataFrame()

    for county in target_df['COUNTY_NAME'].unique():
    # for county in ['Alameda']:
        print(county)

        ##### get sub-dataframe of TAZs within a county
        county_df = df_unscaled.loc[df_unscaled.COUNTY_NAME == county]
#         display(county_df.sort_values(attr_name, ascending=False).head(3))

        ##### adjust total employment
        # calculate adjusted tot employment of all TAZs within the county
        county_df.loc[:, attr_name] = county_df[attr_name] * county_df[scale_ratio_field]

        # round to the nearest integer
        county_df.loc[:, attr_name] = county_df[attr_name].apply(lambda x: int(round(x)))
#         display(county_df.sort_values(attr_name, ascending=False).head(3))

        # correct for rounding errors by allocating the diff to the TAZ with the largest TOTEMP
        target = target_df.loc[target_df.COUNTY_NAME == county][attr_name_in_target_df].sum()
        rounding_diff = target - county_df[attr_name].sum()
#         print(county_df[attr_name].sum(), rounding_diff)
        county_df.loc[county_df[attr_name] == county_df[attr_name].max(),
                           attr_name] = county_df[attr_name] + rounding_diff
#         display(county_df.sort_values(attr_name, ascending=False).head(3))
        
        df_scaled = pd.concat([df_scaled, county_df])
    
    # drop the scale ratio field
    df_scaled.drop(columns=scale_ratio_field, inplace=True)
    
    return df_scaled

In [20]:
def scale_by_taz_and_category(df_cat_unscaled, target_df, attr_tot_name, attr_cat_names, scale_ratio_field):
    """
    Scales a set of sub-category attributes (e.g. 'HHINCQ1', 'HHINCQ2', 'HHINCQ3', 'HHINCQ4') at the TAZ level based on
    county-level scale ratios, and ensures that the sum of the sub-category values for each TAZ equals the total attribute (e.g. TOTHH).
    Three steps:
     - apply the county-level scale ratio to each sub-category attribute for all TAZs within each county
     - round to the nearest interger
     - within each TAZ, correct rounding error by modifying the value of the largest sub-category
    
    Arguments:
        df_cat_unscaled:        e.g. baus_taz_hh_income_unscaled
        target_df:              e.g. baus_taz_tot_hh_scaled
        attr_tot_name:          e.g. 'TOTHH'
        attr_cat_names:         e.g. ['HHINCQ1', 'HHINCQ2', 'HHINCQ3', 'HHINCQ4']
        scale_ratio_field:      e.g. 'TOTHH_pums_baus_ratio'

    """
    
    df_scaled = pd.DataFrame()

    for county in target_df['COUNTY_NAME'].unique():
    # for county in ['Alameda']:
        print(county)

        # get sub-dataframe of TAZs within a county
        county_df = df_cat_unscaled.loc[df_cat_unscaled.COUNTY_NAME == county]
#         display(county_df.head(3))

        # apply the scale ratio to all sub-categories and round to the nearest integer
        for i in attr_cat_names:
            county_df[i].fillna(0, inplace=True)
            county_df.loc[:, i] = county_df[i] * county_df[scale_ratio_field]
            county_df.loc[:, i] = county_df[i].apply(lambda x: int(round(x)))
    
        # correct for rounding errors within each TAZ by allocating the diff to the largest sub-category
        # 1. merge in the scaled total values of the sub-categories
#         print(county_df.shape[0])
        county_df = county_df.merge(target_df, on=['TAZ', 'COUNTY_NAME'], how='left')
#         display(county_df.head())
#         print(county_df.shape[0])
        
        # 2. calculate rounding diff
        county_df.loc[:, 'tot_temp'] = county_df[attr_cat_names].sum(axis=1)
        county_df['rounding_diff'] = county_df[attr_tot_name] - county_df['tot_temp']

        # 3. get the employment values of the largest employment category before rounding error correction 
        largest_cat_values = county_df[attr_cat_names].max(axis=1)
#         print(largest_cat_values)
        # 4. calculate the employment values of the largest employment category with rounding error correction    
        county_df['rounding_adj'] = largest_cat_values + county_df['rounding_diff']
        # 5. get the name of the largest employment category for each TAZ
#         display(county_df[['TAZ', attr_tot_name] + attr_cat_names].max(axis=1))
        county_df['largest_cat'] = county_df[attr_cat_names].idxmax(axis=1)
        # 6. loop through each TAZ to correct the employment value of the largest employment group
#         display(county_df[['TAZ', attr_tot_name] + attr_cat_names + ['tot_temp','rounding_adj','largest_cat']].head())
        for i in county_df.index:
            county_df.loc[i, county_df['largest_cat'][i]] = county_df.loc[i, 'rounding_adj']
#         display(county_df[['TAZ', attr_tot_name] + attr_cat_names + ['tot_temp','rounding_adj','largest_cat']].head())

        df_scaled = pd.concat([df_scaled, county_df])

    # drop the scale ratio field
    df_scaled.drop(columns=[attr_tot_name, scale_ratio_field, 'tot_temp', 'rounding_diff', 'rounding_adj', 'largest_cat'], inplace=True)
    
    return df_scaled

In [21]:
# scale total household counts

attr_name = 'TOTHH'
scale_ratio_field = 'TOTHH_pums_baus_ratio'
df_unscaled = baus_taz_hh_unscaled[['TAZ', 'COUNTY_NAME', 'TOTHH', 'TOTHH_pums_baus_ratio']]
target_df = tot_hh_pums
attr_name_in_target_df = 'TOTHH_pums'

baus_taz_tot_hh_scaled = scale_by_taz(df_unscaled, target_df, attr_name, attr_name_in_target_df, scale_ratio_field)

Alameda
Contra Costa
Marin
Napa
San Francisco
San Mateo
Santa Clara
Solano
Sonoma


In [22]:
baus_taz_hh_unscaled[['COUNTY_NAME', 'TOTHH']].groupby('COUNTY_NAME')['TOTHH'].sum()

COUNTY_NAME
Alameda          574580.0
Contra Costa     388126.0
Marin            108118.0
Napa              51267.0
San Francisco    388605.0
San Mateo        271093.0
Santa Clara      651092.0
Solano           143031.0
Sonoma           188731.0
Name: TOTHH, dtype: float64

In [23]:
baus_taz_tot_hh_scaled[['COUNTY_NAME', 'TOTHH']].groupby('COUNTY_NAME')['TOTHH'].sum()

COUNTY_NAME
Alameda          565858
Contra Costa     411225
Marin            105612
Napa              47926
San Francisco    346357
San Mateo        263750
Santa Clara      637782
Solano           155639
Sonoma           191539
Name: TOTHH, dtype: int64

In [24]:
# scale households by income data

df_cat_unscaled = baus_taz_hh_unscaled[['TAZ', 'COUNTY_NAME', 'HHINCQ1', 'HHINCQ2', 'HHINCQ3', 'HHINCQ4', 'TOTHH_pums_baus_ratio']]
target_df = baus_taz_tot_hh_scaled
attr_tot_name = 'TOTHH'
attr_cat_names = ['HHINCQ1', 'HHINCQ2', 'HHINCQ3', 'HHINCQ4']
scale_ratio_field = 'TOTHH_pums_baus_ratio'

baus_taz_hh_income_scaled = scale_by_taz_and_category(df_cat_unscaled, target_df, attr_tot_name, attr_cat_names, scale_ratio_field)

Alameda
Contra Costa
Marin
Napa
San Francisco
San Mateo
Santa Clara
Solano
Sonoma


In [25]:
# scale households by size data

df_cat_unscaled = baus_taz_hh_unscaled[['TAZ', 'COUNTY_NAME', 'hh_size_1', 'hh_size_2', 'hh_size_3', 'hh_size_4_plus', 'TOTHH_pums_baus_ratio']]
target_df = baus_taz_tot_hh_scaled
attr_tot_name = 'TOTHH'
attr_cat_names = ['hh_size_1', 'hh_size_2', 'hh_size_3', 'hh_size_4_plus']
scale_ratio_field = 'TOTHH_pums_baus_ratio'

baus_taz_hh_size_scaled = scale_by_taz_and_category(df_cat_unscaled, target_df, attr_tot_name, attr_cat_names, scale_ratio_field)

Alameda
Contra Costa
Marin
Napa
San Francisco
San Mateo
Santa Clara
Solano
Sonoma


In [26]:
# scale households by worker count data
df_cat_unscaled = baus_taz_hh_unscaled[['TAZ', 'COUNTY_NAME', 'hh_wrks_0', 'hh_wrks_1', 'hh_wrks_2', 'hh_wrks_3_plus', 'TOTHH_pums_baus_ratio']]
target_df = baus_taz_tot_hh_scaled
attr_tot_name = 'TOTHH'
attr_cat_names = ['hh_wrks_0', 'hh_wrks_1', 'hh_wrks_2', 'hh_wrks_3_plus']
scale_ratio_field = 'TOTHH_pums_baus_ratio'

baus_taz_hh_worker_scaled = scale_by_taz_and_category(df_cat_unscaled, target_df, attr_tot_name, attr_cat_names, scale_ratio_field)

Alameda
Contra Costa
Marin
Napa
San Francisco
San Mateo
Santa Clara
Solano
Sonoma


In [27]:
# scale households by kids data
df_cat_unscaled = baus_taz_hh_unscaled[['TAZ', 'COUNTY_NAME', 'hh_kids_no', 'hh_kids_yes', 'TOTHH_pums_baus_ratio']]
target_df = baus_taz_tot_hh_scaled
attr_tot_name = 'TOTHH'
attr_cat_names = ['hh_kids_no', 'hh_kids_yes']
scale_ratio_field = 'TOTHH_pums_baus_ratio'

baus_taz_hh_kids_scaled = scale_by_taz_and_category(df_cat_unscaled, target_df, attr_tot_name, attr_cat_names, scale_ratio_field)

Alameda
Contra Costa
Marin
Napa
San Francisco
San Mateo
Santa Clara
Solano
Sonoma


In [28]:
# put all household fields together
baus_taz_hh_scaled = baus_taz_tot_hh_scaled.merge(
                     baus_taz_hh_income_scaled, on=['TAZ', 'COUNTY_NAME'], how='outer').merge(
                     baus_taz_hh_size_scaled, on=['TAZ', 'COUNTY_NAME'], how='outer').merge(
                     baus_taz_hh_worker_scaled, on=['TAZ', 'COUNTY_NAME'], how='outer').merge(
                     baus_taz_hh_kids_scaled, on=['TAZ', 'COUNTY_NAME'], how='outer')

baus_taz_hh_scaled

,TAZ,COUNTY_NAME,TOTHH,HHINCQ1,HHINCQ2,HHINCQ3,HHINCQ4,hh_size_1,hh_size_2,hh_size_3,hh_size_4_plus,hh_wrks_0,hh_wrks_1,hh_wrks_2,hh_wrks_3_plus,hh_kids_no,hh_kids_yes
0,715,Alameda,2494,451,490,714,839,286,681,506,1021,222,863,906,503,1333,1161
1,716,Alameda,1669,438,361,428,442,337,530,291,511,391,427,674,177,1106,563
2,717,Alameda,2121,478,465,536,642,351,704,385,681,436,656,718,311,1364,757
3,718,Alameda,2657,467,498,729,963,357,835,519,946,422,627,1117,491,1588,1069
4,719,Alameda,2483,439,542,714,788,496,723,493,771,387,780,971,345,1552,931
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1449,1399,Sonoma,1562,504,408,364,286,353,498,200,511,386,488,477,211,1039,523
1450,1400,Sonoma,1279,162,287,391,439,254,593,164,268,374,357,360,188,1019,260
1451,1401,Sonoma,1554,276,441,436,401,265,647,248,394,356,567,521,110,1154,400
1452,1402,Sonoma,4137,1164,1105,996,872,786,1490,641,1220,1217,1014,1300,606,2835,1302


## 2. employment - compare BAUS with ESRI

### 2.1 ESRI employment data

In [29]:
# TAZ-level employment data from ESRI Business Analyst, scaled to match 2020 regional control totals
esri_emp_taz = pd.read_csv(ESRI_EMP_TAZ_FILE)
esri_emp_taz = esri_emp_taz[['TAZ1454', 'County_Name', 
                                           'TOTEMP', 'AGREMPN', 'FPSEMPN', 'HEREMPN', 'MWTEMPN', 'OTHEMPN', 'RETEMPN']]
display(esri_emp_taz.head())
esri_emp_taz.columns = ['TAZ1454', 'COUNTY_NAME'] + [x + '_esri' for x in list(esri_emp_taz)[2:]]

display(esri_emp_taz.head())

,TAZ1454,County_Name,TOTEMP,AGREMPN,FPSEMPN,HEREMPN,MWTEMPN,OTHEMPN,RETEMPN
0,1,San Francisco,17550.632095,25.770208,12023.482700,2763.014476,422.407323,1872.261634,443.695755
1,2,San Francisco,27832.945089,23.529320,20407.763852,3975.334696,717.084049,2327.161827,382.071345
2,3,San Francisco,2421.279109,0.000000,1001.676781,1037.530983,50.419972,211.763883,119.887489
3,4,San Francisco,16246.435481,23.529320,10685.672771,1980.944685,685.711622,2006.714893,863.862190
4,5,San Francisco,16787.609849,6.722663,4138.919494,6533.307951,534.451705,966.943022,4607.265014


,TAZ1454,COUNTY_NAME,TOTEMP_esri,AGREMPN_esri,FPSEMPN_esri,HEREMPN_esri,MWTEMPN_esri,OTHEMPN_esri,RETEMPN_esri
0,1,San Francisco,17550.632095,25.770208,12023.482700,2763.014476,422.407323,1872.261634,443.695755
1,2,San Francisco,27832.945089,23.529320,20407.763852,3975.334696,717.084049,2327.161827,382.071345
2,3,San Francisco,2421.279109,0.000000,1001.676781,1037.530983,50.419972,211.763883,119.887489
3,4,San Francisco,16246.435481,23.529320,10685.672771,1980.944685,685.711622,2006.714893,863.862190
4,5,San Francisco,16787.609849,6.722663,4138.919494,6533.307951,534.451705,966.943022,4607.265014


In [30]:
# get total employment by county
esribiz_emptot_county = esri_emp_taz.groupby('COUNTY_NAME')['TOTEMP_esri'].sum().reset_index()
display(esribiz_emptot_county)

,COUNTY_NAME,TOTEMP_esri
0,Alameda,8.270713e+05
1,Contra Costa,4.253429e+05
2,Marin,1.394011e+05
3,Napa,9.275370e+04
4,San Francisco,6.683257e+05
5,San Mateo,4.268331e+05
6,Santa Clara,1.109100e+06
7,Solano,1.584285e+05
8,Sonoma,2.321492e+05


### 2.2 compare BAUS total employment by county with ESRI

In [31]:
# get BAUS 2020 output total employment by county
baus_emptot_county = baus_taz.groupby('COUNTY_NAME')[['TOTEMP']].sum().reset_index().rename(
    columns={'TOTEMP': 'TOTEMP_baus'})
display(baus_emptot_county)

,COUNTY_NAME,TOTEMP_baus
0,Alameda,898270.0
1,Contra Costa,432854.0
2,Marin,136991.0
3,Napa,68988.0
4,San Francisco,675180.0
5,San Mateo,393475.0
6,Santa Clara,1114678.0
7,Solano,134092.0
8,Sonoma,224878.0


In [32]:
# merge ESRI with BAUS
emptot_compare = esribiz_emptot_county.merge(baus_emptot_county, on='COUNTY_NAME', how='left')

# convert 'TOTEMP_esri' to integer
emptot_compare['TOTEMP_esri'] = emptot_compare['TOTEMP_esri'].apply(lambda x: int(round(x)))

print('total employment comparison:\n{}'.format(emptot_compare[['TOTEMP_esri', 'TOTEMP_baus']].sum()))

# correct rounding error by adjusting the largest employment county
rounding_diff = emptot_compare['TOTEMP_baus'].sum() - emptot_compare['TOTEMP_esri'].sum()
emptot_compare.loc[emptot_compare.TOTEMP_esri == emptot_compare.TOTEMP_esri.max(),
                   'TOTEMP_esri'] = emptot_compare['TOTEMP_esri'] + rounding_diff
# check the totals match
print('after correcting for rounding error, total employment comparison:\n{}'.format(emptot_compare[['TOTEMP_esri', 'TOTEMP_baus']].sum()))

# add esri / baus ratio by county
emptot_compare['totemp_esri_baus_diff'] = emptot_compare['TOTEMP_esri'] - emptot_compare['TOTEMP_baus']
emptot_compare['totemp_esri_baus_ratio'] = emptot_compare['TOTEMP_esri'] / emptot_compare['TOTEMP_baus']

display(emptot_compare.sort_values('totemp_esri_baus_diff', ascending=False))

total employment comparison:
TOTEMP_esri    4079406.0
TOTEMP_baus    4079406.0
dtype: float64
after correcting for rounding error, total employment comparison:
TOTEMP_esri    4079406.0
TOTEMP_baus    4079406.0
dtype: float64


,COUNTY_NAME,TOTEMP_esri,TOTEMP_baus,totemp_esri_baus_diff,totemp_esri_baus_ratio
5,San Mateo,426833.0,393475.0,33358.0,1.084778
7,Solano,158429.0,134092.0,24337.0,1.181495
3,Napa,92754.0,68988.0,23766.0,1.344495
8,Sonoma,232149.0,224878.0,7271.0,1.032333
2,Marin,139401.0,136991.0,2410.0,1.017592
6,Santa Clara,1109100.0,1114678.0,-5578.0,0.994996
4,San Francisco,668326.0,675180.0,-6854.0,0.989849
1,Contra Costa,425343.0,432854.0,-7511.0,0.982648
0,Alameda,827071.0,898270.0,-71199.0,0.920738


### 2.3 scale BAUS employment to be consistent with ESRI data at the county level

In [33]:
# calculate scale ratio by county
emp_adjust_ratio = emptot_compare[['COUNTY_NAME', 'totemp_esri_baus_ratio']]

baus_taz_emp_unscaled = baus_taz[['TAZ', 'COUNTY_NAME', 'AGREMPN', 'FPSEMPN', 'HEREMPN', 'RETEMPN', 'MWTEMPN', 'OTHEMPN', 'TOTEMP']].merge(emp_adjust_ratio, on='COUNTY_NAME', how='left')
baus_taz_emp_unscaled

,TAZ,COUNTY_NAME,AGREMPN,FPSEMPN,HEREMPN,RETEMPN,MWTEMPN,OTHEMPN,TOTEMP,totemp_esri_baus_ratio
0,1,San Francisco,10.0,13984.0,1089.0,219.0,648.0,1702.0,17652.0,0.989849
1,2,San Francisco,34.0,18629.0,2116.0,378.0,582.0,1847.0,23586.0,0.989849
2,3,San Francisco,1.0,1469.0,1390.0,158.0,39.0,301.0,3358.0,0.989849
3,4,San Francisco,82.0,12682.0,3842.0,486.0,1378.0,1959.0,20429.0,0.989849
4,5,San Francisco,5.0,5318.0,5746.0,2763.0,399.0,1742.0,15973.0,0.989849
...,...,...,...,...,...,...,...,...,...,...
1449,1450,Marin,1.0,273.0,363.0,212.0,31.0,168.0,1048.0,1.017592
1450,1451,Marin,8.0,251.0,230.0,80.0,30.0,126.0,725.0,1.017592
1451,1452,Marin,3.0,611.0,657.0,266.0,110.0,428.0,2075.0,1.017592
1452,1453,Marin,0.0,200.0,391.0,4.0,25.0,302.0,922.0,1.017592


In [34]:
# scale total employment
attr_name = 'TOTEMP'
scale_ratio_field = 'totemp_esri_baus_ratio'
df_unscaled = baus_taz_emp_unscaled[['TAZ', 'COUNTY_NAME', 'TOTEMP', 'totemp_esri_baus_ratio']]
target_df = emptot_compare[['COUNTY_NAME', 'TOTEMP_esri']]
attr_name_in_target_df = 'TOTEMP_esri'

baus_taz_tot_emp_scaled = scale_by_taz(df_unscaled, target_df, attr_name, attr_name_in_target_df, scale_ratio_field)

Alameda
Contra Costa
Marin
Napa
San Francisco
San Mateo
Santa Clara
Solano
Sonoma


In [35]:
# scale employment by type
df_cat_unscaled = baus_taz_emp_unscaled[['TAZ', 'COUNTY_NAME', 'AGREMPN', 'FPSEMPN', 'HEREMPN', 'RETEMPN', 'MWTEMPN', 'OTHEMPN', 'totemp_esri_baus_ratio']]
target_df = baus_taz_tot_emp_scaled
attr_tot_name = 'TOTEMP'
attr_cat_names = ['AGREMPN', 'FPSEMPN', 'HEREMPN', 'RETEMPN', 'MWTEMPN', 'OTHEMPN']
scale_ratio_field = 'totemp_esri_baus_ratio'

baus_taz_emp_type_scaled = scale_by_taz_and_category(df_cat_unscaled, target_df, attr_tot_name, attr_cat_names, scale_ratio_field)

Alameda
Contra Costa
Marin
Napa
San Francisco
San Mateo
Santa Clara
Solano
Sonoma


In [36]:
# put all employment fields together
baus_taz_emp_scaled = baus_taz_tot_emp_scaled.merge(baus_taz_emp_type_scaled, on=['TAZ', 'COUNTY_NAME'], how='outer')
baus_taz_emp_scaled

,TAZ,COUNTY_NAME,TOTEMP,AGREMPN,FPSEMPN,HEREMPN,RETEMPN,MWTEMPN,OTHEMPN
0,715,Alameda,16063.0,131.0,7299.0,1294.0,502.0,3188.0,3649.0
1,716,Alameda,2734.0,10.0,845.0,1020.0,91.0,114.0,654.0
2,717,Alameda,1591.0,0.0,123.0,696.0,93.0,160.0,519.0
3,718,Alameda,4152.0,18.0,579.0,385.0,438.0,1099.0,1633.0
4,719,Alameda,360.0,0.0,87.0,87.0,48.0,25.0,113.0
...,...,...,...,...,...,...,...,...,...
1449,1399,Sonoma,6391.0,45.0,664.0,1721.0,1432.0,1578.0,951.0
1450,1400,Sonoma,1806.0,426.0,201.0,368.0,66.0,337.0,408.0
1451,1401,Sonoma,3848.0,805.0,308.0,1144.0,137.0,739.0,715.0
1452,1402,Sonoma,3739.0,524.0,408.0,934.0,304.0,638.0,931.0


## 3. employed residents

### 3.1 PUMS person data

In [37]:
# employment data in PUMS person file

p_pums_2020 = pd.read_csv(PUMS_P_FILE, usecols = [
    'PUMA', 
    'County_Name',
    'PWGTP',   # Person's weight:
#               1..9999 .Integer weight of person 
    'ESR',     # Employment status recode:
#               b .N/A (less than 16 years old)
#               1 .Civilian employed, at work
#               2 .Civilian employed, with a job but not at work
#               3 .Unemployed
#               4 .Armed forces, at work
#               5 .Armed forces, with a job but not at work
#               6 .Not in labor force
])
p_pums_2020.rename(columns={'County_Name': 'COUNTY_NAME'}, inplace=True)
display(p_pums_2020)

,PUMA,PWGTP,ESR,COUNTY_NAME
0,101,95,1.0,Alameda
1,101,65,1.0,Alameda
2,101,59,1.0,Alameda
3,101,39,6.0,Alameda
4,101,91,1.0,Alameda
...,...,...,...,...
63144,9703,90,6.0,Sonoma
63145,9703,133,6.0,Sonoma
63146,9703,87,6.0,Sonoma
63147,9703,84,1.0,Sonoma


In [38]:
# recode ESR: non employed categories to 0, employed categories to 1 
employed_dict = {1: 1,
                 2: 1,
                 3: 0,
                 4: 1,
                 5: 1,
                 6: 0}

p_pums_2020['employed_recode'] = p_pums_2020['ESR'].map(employed_dict)
p_pums_2020['employed_recode'].fillna(0)
p_pums_2020['EMPRES_pums'] = p_pums_2020['employed_recode'] * p_pums_2020['PWGTP']
pums_empres_county = p_pums_2020.groupby('COUNTY_NAME')['EMPRES_pums'].sum().reset_index()

In [39]:
print(pums_empres_county.EMPRES_pums.sum())
pums_empres_county

3893394.0


,COUNTY_NAME,EMPRES_pums
0,Alameda,843118.0
1,Contra Costa,543589.0
2,Marin,124225.0
3,Napa,65331.0
4,San Francisco,493622.0
5,San Mateo,388176.0
6,Santa Clara,985917.0
7,Solano,208469.0
8,Sonoma,240947.0


### 3.2 compare with BAUS 'EMPRES'

In [40]:
# get BAUS 2020 output 'EMPRES' by county
baus_empres_county = baus_taz.groupby('COUNTY_NAME')[['EMPRES']].sum().reset_index().rename(
    columns={'EMPRES': 'EMPRES_baus'})
display(baus_empres_county)

,COUNTY_NAME,EMPRES_baus
0,Alameda,891400.0
1,Contra Costa,608490.0
2,Marin,136528.0
3,Napa,82521.0
4,San Francisco,547495.0
5,San Mateo,412057.0
6,Santa Clara,1011751.0
7,Solano,236531.0
8,Sonoma,285516.0


In [41]:
# merge ESRI with BAUS
empres_compare = pums_empres_county.merge(baus_empres_county, on='COUNTY_NAME', how='left')

print('total employment withno incommute comparison:\n{}'.format(empres_compare[['EMPRES_pums', 'EMPRES_baus']].sum()))

# add esri / baus ratio by county
empres_compare['empres_pums_baus_diff'] = empres_compare['EMPRES_pums'] - empres_compare['EMPRES_baus']
empres_compare['empres_pums_baus_ratio'] = empres_compare['EMPRES_pums'] / empres_compare['EMPRES_baus']

display(empres_compare.sort_values('empres_pums_baus_diff', ascending=False))

total employment withno incommute comparison:
EMPRES_pums    3893394.0
EMPRES_baus    4212289.0
dtype: float64


,COUNTY_NAME,EMPRES_pums,EMPRES_baus,empres_pums_baus_diff,empres_pums_baus_ratio
2,Marin,124225.0,136528.0,-12303.0,0.909887
3,Napa,65331.0,82521.0,-17190.0,0.791689
5,San Mateo,388176.0,412057.0,-23881.0,0.942044
6,Santa Clara,985917.0,1011751.0,-25834.0,0.974466
7,Solano,208469.0,236531.0,-28062.0,0.881360
8,Sonoma,240947.0,285516.0,-44569.0,0.843900
0,Alameda,843118.0,891400.0,-48282.0,0.945836
4,San Francisco,493622.0,547495.0,-53873.0,0.901601
1,Contra Costa,543589.0,608490.0,-64901.0,0.893341


### 3.3 scale BAUS EMPRES by county

In [42]:
# calculate scale ratio by county
empres_adjust_ratio = empres_compare[['COUNTY_NAME', 'empres_pums_baus_ratio']]

baus_taz_empres_unscaled = baus_taz[['TAZ', 'COUNTY_NAME', 'EMPRES']].merge(empres_adjust_ratio, on='COUNTY_NAME', how='left')
baus_taz_empres_unscaled

,TAZ,COUNTY_NAME,EMPRES,empres_pums_baus_ratio
0,1,San Francisco,41.0,0.901601
1,2,San Francisco,119.0,0.901601
2,3,San Francisco,229.0,0.901601
3,4,San Francisco,152.0,0.901601
4,5,San Francisco,507.0,0.901601
...,...,...,...,...
1449,1450,Marin,4170.0,0.909887
1450,1451,Marin,2783.0,0.909887
1451,1452,Marin,2458.0,0.909887
1452,1453,Marin,366.0,0.909887


In [43]:
# scale total employment
attr_name = 'EMPRES'
scale_ratio_field = 'empres_pums_baus_ratio'
df_unscaled = baus_taz_empres_unscaled[['TAZ', 'COUNTY_NAME', 'EMPRES', 'empres_pums_baus_ratio']]
target_df = pums_empres_county
attr_name_in_target_df = 'EMPRES_pums'

baus_taz_empres_scaled = scale_by_taz(df_unscaled, target_df, attr_name, attr_name_in_target_df, scale_ratio_field)
baus_taz_empres_scaled

Alameda
Contra Costa
Marin
Napa
San Francisco
San Mateo
Santa Clara
Solano
Sonoma


,TAZ,COUNTY_NAME,EMPRES
714,715,Alameda,4389.0
715,716,Alameda,1926.0
716,717,Alameda,2849.0
717,718,Alameda,4337.0
718,719,Alameda,3622.0
...,...,...,...
1398,1399,Sonoma,1887.0
1399,1400,Sonoma,2019.0
1400,1401,Sonoma,2535.0
1401,1402,Sonoma,4151.0


In [44]:
baus_taz_empres_scaled.groupby('COUNTY_NAME')['EMPRES'].sum()

COUNTY_NAME
Alameda          843118.0
Contra Costa     543589.0
Marin            124225.0
Napa              65331.0
San Francisco    493622.0
San Mateo        388176.0
Santa Clara      985917.0
Solano           208469.0
Sonoma           240947.0
Name: EMPRES, dtype: float64

## 4. recalculate densities

In [45]:
# BAUS TAZ data after scaling
baus_taz_scaled = baus_taz_hh_scaled.merge(
                  baus_taz_emp_scaled, on=['TAZ', 'COUNTY_NAME'], how='outer').merge(
                  baus_taz[id_fields + pop_fields + housing_fields + land_fields], on=['TAZ', 'COUNTY_NAME'], how='outer').merge(
                  baus_taz_empres_scaled)



In [46]:
# calculate density fields
baus_taz_scaled['DENSITY_POP'] = baus_taz_scaled.TOTPOP / baus_taz_scaled.TOTACRE
baus_taz_scaled['DENSITY_POP'].fillna(0, inplace=True)

baus_taz_scaled['DENSITY_EMP'] = (2.5 * baus_taz_scaled.TOTEMP) / baus_taz_scaled.TOTACRE
baus_taz_scaled['DENSITY_EMP'].fillna(0, inplace=True)

baus_taz_scaled['DENSITY'] = baus_taz_scaled['DENSITY_POP'] + baus_taz_scaled['DENSITY_EMP']
baus_taz_scaled['AREATYPE'] = pd.cut(
    baus_taz_scaled.DENSITY,
    bins=[0, 6, 30, 55, 100, 300, np.inf],
    labels=[5, 4, 3, 2, 1, 0]
)

In [47]:
baus_taz_scaled

,TAZ,COUNTY_NAME,TOTHH,HHINCQ1,HHINCQ2,HHINCQ3,HHINCQ4,hh_size_1,hh_size_2,hh_size_3,...,TOTACRE,RESACRE_UNWEIGHTED,CIACRE_UNWEIGHTED,CIACRE,RESACRE,EMPRES,DENSITY_POP,DENSITY_EMP,DENSITY,AREATYPE
0,715,Alameda,2494,451,490,714,839,286,681,506,...,137540.0,72430.713047,7724.523430,3480.022526,5215.494171,4389.0,0.059154,0.291970,0.351123,5
1,716,Alameda,1669,438,361,428,442,337,530,291,...,1182.0,788.335043,210.242763,129.762350,970.000000,1926.0,4.032149,5.782572,9.814721,4
2,717,Alameda,2121,478,465,536,642,351,704,385,...,677.0,319.080781,87.752544,151.000000,494.561315,2849.0,9.251108,5.875185,15.126292,4
3,718,Alameda,2657,467,498,729,963,357,835,519,...,1344.0,455.353585,305.958998,268.000000,817.000000,4337.0,6.055804,7.723214,13.779018,4
4,719,Alameda,2483,439,542,714,788,496,723,493,...,911.0,386.387725,44.386105,37.729440,520.147352,3622.0,7.731065,0.987925,8.718990,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1449,1399,Sonoma,1562,504,408,364,286,353,498,200,...,791.0,237.849683,322.333603,499.000000,274.629761,1887.0,5.785082,20.199115,25.984197,4
1450,1400,Sonoma,1279,162,287,391,439,254,593,164,...,55176.0,4423.246038,22190.149044,1309.000000,3623.735242,2019.0,0.057724,0.081829,0.139553,5
1451,1401,Sonoma,1554,276,441,436,401,265,647,248,...,142961.0,5115.125324,60923.637170,2348.000000,4194.776109,2535.0,0.030036,0.067291,0.097327,5
1452,1402,Sonoma,4137,1164,1105,996,872,786,1490,641,...,53379.0,2312.231583,24456.086664,1689.271714,3219.776260,4151.0,0.215834,0.175116,0.390950,5


In [48]:
# check the fields are the same as BAUS output
sorted(list(baus_taz_scaled)) == sorted(list(baus_taz))

True

## 5. export

In [49]:
baus_taz_scaled.to_csv(taz_summaries_scaled, index=False)